# Clustering

In [1]:
# ! conda install geopy
# ! conda install shapely
# ! pip3 install geopy
#! pip3 install shapely

In [2]:
import numpy as np
import pandas as pd
import os

from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [3]:
# data from previous notebook
df_event = pd.read_csv('event_data.csv')
df_non_event = pd.read_csv('non_event_data.csv')
df_event.device_id.count(),df_non_event.device_id.count() #(1215598, 74645)

C:\Users\chand\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1215598, 74645)

In [4]:
df_invalid_location = df_event[((df_event.latitude <= 1) | (df_event.longitude <= 1))]
df_valid_location = df_event[~((df_event.latitude <= 1) | (df_event.longitude <= 1))]
df_invalid_location.device_id.count(),df_valid_location.device_id.count() #(385966, 829632)

(385966, 829632)

In [5]:
df_valid_location.head()

,device_id,gender,age,group,event_id,timestamp,longitude,latitude,has_event,week_day,hour_day,week,year,group_cat,hour_day_cat
0,-7548291590301750000,M,33,M32+,414.0,2016-05-03 00:05:31,116.79,33.98,1,Tuesday,0,18,2016,33-45,Night
1,-7548291590301750000,M,33,M32+,2604805.0,2016-05-04 03:01:32,116.79,33.98,1,Wednesday,3,18,2016,33-45,Night
2,-7548291590301750000,M,33,M32+,2588911.0,2016-05-03 16:14:14,116.79,33.98,1,Tuesday,16,18,2016,33-45,Afternoon
3,-7548291590301750000,M,33,M32+,2586055.0,2016-05-02 14:36:04,116.79,33.98,1,Monday,14,18,2016,33-45,Afternoon
4,-7548291590301750000,M,33,M32+,2560938.0,2016-05-02 18:01:00,116.79,33.98,1,Monday,18,18,2016,33-45,Afternoon


In [6]:
#Loading the Data 
coords = df_valid_location[['longitude', 'latitude']]

In [7]:
kms_per_radian = 6371.0088 # this is a constant
epsilon = 1.5/kms_per_radian 
db = DBSCAN(eps= epsilon, min_samples= 10, algorithm = 'ball_tree', metric = 'haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coords[cluster_labels ==n] for n in range(num_clusters)])
print('Number of clusters : {}'.format(num_clusters))
print('Number of cluster label countsclusters : {}'.format(len(cluster_labels)))

Number of clusters : 3639
Number of cluster label countsclusters : 829632


In [8]:
# check records with label  = 15
#coords[cluster_labels ==15]

In [9]:
df_valid_location['loc_cluster'] = [i for i in cluster_labels]

C:\Users\chand\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_invalid_location['loc_cluster'] = 'Unknown'

C:\Users\chand\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_event = pd.concat([df_valid_location,df_invalid_location])

In [12]:
df_cat_only = df_event[['device_id','loc_cluster']]

In [13]:
# merge the data with non_event_dataset
df_non_event=df_non_event.merge(df_cat_only, how='left', on='device_id')
df_non_event.tail()

,device_id,gender,age,group,phone_brand,device_model,group_cat,avg_events,most_active_hour,loc_chng_morning,loc_chng_afternoon,loc_chng_evening,loc_chng_night,loc_cluster
1266928,-8270585312108800000,F,32,F25-32,OPPO,U707T,25-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266929,9140950698473710000,M,41,M32+,Huawei,Mate 8,33-45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266930,-5051737733034250000,M,25,M25-32,Xiaomi,x1,25-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266931,-6901678500015010000,F,20,F0-24,vivo,X3L,0-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266932,6076451050607320000,M,21,M0-24,Xiaomi,MI 3,0-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Observation: Based on DBSCAN clustering, We have found that there are total of 7231, clusters based on longitude, latitude. We also created a special cluster for missing lat/long data.

In [14]:
#save the dataset for next notebook
df_event.to_csv('../4_Data_Prepration/event_data.csv',index=False)
df_non_event.to_csv('../4_Data_Prepration/non_event_data.csv',index=False)

In [15]:
#delete the dataframes
del df_event
del df_non_event

----

End of this notebook